In [1]:
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter  
import math 
from scipy import stats

In [3]:
stocks = pd.read_csv('starter-files/sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2898562939467,
 'week52high': 179.14,
 'week52low': 115.85,
 'week52highSplitAdjustOnly': 177.35,
 'week52lowSplitAdjustOnly': 121.62,
 'week52change': 0.4660238074040665,
 'sharesOutstanding': 16563590791,
 'float': 0,
 'avg10Volume': 137365788,
 'avg30Volume': 101877283,
 'day200MovingAvg': 148.05,
 'day50MovingAvg': 162.56,
 'employees': 151834,
 'ttmEPS': 11.32,
 'ttmDividendRate': 0.8884534874101977,
 'dividendYield': 0.004951207925995745,
 'nextDividendDate': '',
 'exDividendDate': '2021-11-02',
 'nextEarningsDate': '2022-01-12',
 'peRatio': 15.61041523803717,
 'beta': 1.3599503061120797,
 'maxChangePercent': 66.645613444051,
 'year5ChangePercent': 5.742291011401765,
 'year2ChangePercent': 1.6610558441423724,
 'year1ChangePercent': 0.43385159402701295,
 'ytdChangePercent': 0.3408679327416336,
 'month6ChangePercent': 0.404280928918315,
 'month3ChangePercent': 0.1312170157451665,
 'month1ChangePercent': 0.16385914732328544,
 'day30ChangePe

In [5]:
data['year1ChangePercent']

0.43385159402701295

In [6]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))


my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:

    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

In [9]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,82.62,2.374185,N/A
1,DVN,42.95,2.009384,N/A
2,FANG,113.44,1.574894,N/A
3,MCHP,89.47,1.485754,N/A
4,FTNT,320.62,1.459994,N/A
5,NVDA,319.30,1.396439,N/A
6,MRO,16.54,1.335394,N/A
7,F,20.17,1.193649,N/A
8,IT,318.77,1.108871,N/A
9,NUE,113.68,1.089602,N/A


In [10]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter competition portfolio size here: ")

try:
    val = float(portfolio_size)
except ValueError:
    print("Enter the data in numericall form")
    portfolio_size = input("Enter competition portfolio size here: ")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:100000000
100000000


In [11]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

c:\users\nilay\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,82.62,2.374185,23732
1,DVN,42.95,2.009384,45652
2,FANG,113.44,1.574894,17284
3,MCHP,89.47,1.485754,21915
4,FTNT,320.62,1.459994,6115
5,NVDA,319.30,1.396439,6140
6,MRO,16.54,1.335394,118548
7,F,20.17,1.193649,97212
8,IT,318.77,1.108871,6151
9,NUE,113.68,1.089602,17248


In [12]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:

    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

In [13]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

   
hqm_dataframe

0      0.637624
1       0.20396
2      0.845545
3       0.70495
4           0.4
         ...   
500    0.532673
501    0.059406
502    0.883168
503     0.80198
504    0.756436
Name: One-Year Return Percentile, Length: 505, dtype: object
0      0.687129
1      0.033663
2       0.80198
3      0.954455
4      0.649505
         ...   
500    0.667327
501     0.10495
502    0.794059
503    0.651485
504    0.918812
Name: Six-Month Return Percentile, Length: 505, dtype: object
0      0.073267
1      0.227723
2      0.891089
3      0.807921
4      0.790099
         ...   
500    0.461386
501    0.077228
502    0.556436
503     0.79802
504    0.732673
Name: Three-Month Return Percentile, Length: 505, dtype: object
0      0.364356
1      0.011881
2      0.710891
3      0.984158
4      0.827723
         ...   
500    0.813861
501    0.170297
502    0.639604
503    0.372277
504    0.845545
Name: One-Month Return Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,155.42,N/A,0.357437,0.637624,0.129979,0.687129,-0.128657,0.073267,-0.028866,0.364356,N/A
1,AAL,18.70,N/A,0.034558,0.20396,-0.250166,0.033663,-0.047262,0.227723,-0.188954,0.011881,N/A
2,AAP,243.70,N/A,0.574883,0.845545,0.202048,0.80198,0.177307,0.891089,0.027263,0.710891,N/A
3,AAPL,174.66,N/A,0.422480,0.70495,0.404205,0.954455,0.131962,0.807921,0.164777,0.984158,N/A
4,ABBV,127.47,N/A,0.192152,0.4,0.111603,0.649505,0.123479,0.790099,0.051316,0.827723,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,135.38,N/A,0.279798,0.532673,0.119505,0.667327,0.016011,0.461386,0.049288,0.813861,N/A
501,ZBH,129.00,N/A,-0.118515,0.059406,-0.180683,0.10495,-0.128149,0.077228,-0.063447,0.170297,N/A
502,ZBRA,600.50,N/A,0.614505,0.883168,0.198745,0.794059,0.045620,0.556436,0.015004,0.639604,N/A
503,ZION,63.76,N/A,0.527868,0.80198,0.111666,0.651485,0.129466,0.79802,-0.025472,0.372277,N/A


In [14]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [15]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

In [16]:
portfolio_input()


Enter the value of your portfolio:100000000


In [17]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

c:\users\nilay\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,155.42,12616,0.357437,0.637624,0.129979,0.687129,-0.128657,0.073267,-0.028866,0.364356,0.440594
1,AAL,18.70,104854,0.034558,0.20396,-0.250166,0.033663,-0.047262,0.227723,-0.188954,0.011881,0.119307
2,AAP,243.70,8045,0.574883,0.845545,0.202048,0.80198,0.177307,0.891089,0.027263,0.710891,0.812376
3,AAPL,174.66,11226,0.422480,0.70495,0.404205,0.954455,0.131962,0.807921,0.164777,0.984158,0.862871
4,ABBV,127.47,15382,0.192152,0.4,0.111603,0.649505,0.123479,0.790099,0.051316,0.827723,0.666832
5,ABC,122.06,16064,0.227272,0.457426,0.032607,0.465347,-0.024859,0.306931,-0.037719,0.29703,0.381683
6,ABMD,323.54,6060,0.187152,0.392079,0.122038,0.675248,-0.138108,0.067327,-0.098545,0.067327,0.300495
7,ABT,136.26,14390,0.266004,0.50297,0.241039,0.853465,0.030595,0.506931,0.044417,0.79802,0.665347
8,ACN,389.52,5033,0.538922,0.821782,0.339005,0.926733,0.105336,0.742574,0.021594,0.683168,0.793564
9,ADBE,636.80,3079,0.328014,0.59604,0.292467,0.89703,-0.015542,0.348515,-0.019713,0.409901,0.562871


In [18]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [19]:
background_color = '#ffffff'
font_color = '#000000'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [20]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [21]:
writer.save()